# Agent Tools and Configurations

Tools give agents ability to take actions: sequential/parallel calls, error handling, and state persistence.

**What you'll learn:**
- Tools extend agent capabilities with actions
- @wrap_tool_call handles errors gracefully
- Agent selects tools based on docstrings
- Sequential vs parallel execution determined by model
- Clear tool descriptions guide selection

In [ ]:
import sys
sys.path.append('D:/Courses/Udemy/AI Agent Projects')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Basic Agent with Tools

In [ ]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather]
)

response = agent.invoke({
    'messages': [HumanMessage("What's the weather in Tokyo?")]
})

response['messages'][-1].text

In [ ]:
# Agent without tools - single LLM node
agent_no_tools = create_agent(model=model, tools=[])

response = agent_no_tools.invoke({
    'messages': [HumanMessage("Hello")]
})

response['messages'][-1].text

## Tool Error Handling

In [14]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from langchain.tools import tool
# Tool WITHOUT internal error handling
@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    return a / b  # This will crash on division by zero

# Middleware catches it and returns graceful error to model
@wrap_tool_call
def handle_tool_errors(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}. Try different input.",
            tool_call_id=request.tool_call["id"]
        )


agent_safe = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather, divide],
    middleware=[handle_tool_errors]
)

response = agent_safe.invoke({
    'messages': [HumanMessage("what is 1/0?")]
})

response['messages'][-1].text

"It looks like you're trying to divide by zero, which is not possible. Would you like to try a different calculation?"

## Sequential Tool Calls

In [ ]:
# Agent calls tools in sequence
response = agent.invoke({
    'messages': [HumanMessage(
        "Search for Apple news, then tell me weather in Cupertino"
    )]
})

response['messages'][-1].text

## Parallel Tool Calls

In [ ]:
# Agent may call tools in parallel (independent requests)
response = agent.invoke({
    'messages': [HumanMessage(
        "What's the weather in Paris and London?"
    )]
})

response['messages'][-1].text

In [ ]:
# Exercise: Test with different queries
